In [4]:
import pandas as pd
from datetime import timedelta

# Load cleaned retail data
df = pd.read_csv('../data/cleaned_retail.csv', parse_dates=['InvoiceDate'])

# Fix inconsistent column names
df.rename(columns={'Customer ID': 'CustomerID'}, inplace=True)

# Ensure valid data
df = df[df['Quantity'] > 0]
df = df[df['CustomerID'].notnull()]
df['CustomerID'] = df['CustomerID'].astype(int)

# Create TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['Price']

# Snapshot date for recency
snapshot_date = df['InvoiceDate'].max() + timedelta(days=1)

# Base RFM features
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'Invoice': 'nunique',
    'TotalPrice': 'sum'
}).reset_index()
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Total products purchased
total_products = df.groupby('CustomerID')['Quantity'].sum().reset_index(name='TotalProductsPurchased')

# Unique products
unique_products = df.groupby('CustomerID')['Description'].nunique().reset_index(name='UniqueProducts')

# Avg basket size
basket = df.groupby('CustomerID').agg({
    'TotalPrice': 'sum',
    'Invoice': 'nunique'
}).reset_index()
basket['AvgBasketSize'] = basket['TotalPrice'] / basket['Invoice']
basket = basket[['CustomerID', 'AvgBasketSize']]

# Avg days between purchases
invoice_dates = df.groupby('CustomerID')['InvoiceDate'].agg(['min', 'max', 'count']).reset_index()
invoice_dates['AvgDaysBetweenPurchases'] = invoice_dates.apply(
    lambda row: (row['max'] - row['min']).days / (row['count'] - 1) if row['count'] > 1 else 0, axis=1)
invoice_dates['DaysSinceFirstPurchase'] = (snapshot_date - invoice_dates['min']).dt.days
invoice_dates = invoice_dates[['CustomerID', 'AvgDaysBetweenPurchases', 'DaysSinceFirstPurchase']]

# Cancellation rate
df['IsCancelled'] = df['Invoice'].astype(str).str.startswith('C').astype(int)
cancellations = df.groupby('CustomerID')['IsCancelled'].mean().reset_index(name='CancellationRate')

# Merge everything
features = rfm.merge(total_products, on='CustomerID', how='left') \
              .merge(unique_products, on='CustomerID', how='left') \
              .merge(basket, on='CustomerID', how='left') \
              .merge(invoice_dates, on='CustomerID', how='left') \
              .merge(cancellations, on='CustomerID', how='left')

# Load segments
segments = pd.read_csv('../data/customer_segments.csv')[['CustomerID', 'Segment']]

# Merge with features
final_df = features.merge(segments, on='CustomerID', how='left')

# Save final dataset for dashboard
final_df.to_csv('../data/final_dashboard_dataset.csv', index=False)
print("✅ Saved: final_dashboard_dataset.csv")


✅ Saved: final_dashboard_dataset.csv
